# Lynx-Hare Predator-Prey Discovery

Discover predator-prey dynamics from real ecological data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sc_sindy import (
    sindy_stls,
    build_library_2d,
    compute_derivatives_spline,
    format_equation,
)

## Load and Visualize Data

In [ ]:
# Load Lynx-Hare data
df = pd.read_csv('../../data/raw/lynx_hare.csv')
print(df.head())
print(f"\nYears: {df['year'].min()} - {df['year'].max()}")

In [ ]:
# Visualize
plt.figure(figsize=(12, 5))
plt.plot(df['year'], df['hare']/1000, 'b-o', label='Hare (thousands)')
plt.plot(df['year'], df['lynx']/1000, 'r-o', label='Lynx (thousands)')
plt.xlabel('Year')
plt.ylabel('Population (thousands)')
plt.title('Hudson Bay Company Lynx-Hare Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Prepare Data for SINDy

In [ ]:
# Normalize data
X = df[['hare', 'lynx']].values
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

print(f"Data shape: {X_norm.shape}")
print(f"Normalized mean: {X_norm.mean(axis=0)}")
print(f"Normalized std: {X_norm.std(axis=0)}")

In [ ]:
# Compute derivatives using splines (smoother for sparse data)
dt = 1.0  # Annual data
X_dot = compute_derivatives_spline(X_norm, dt)

print(f"Derivative shape: {X_dot.shape}")

## Discover Equations

In [ ]:
# Build library
Theta, labels = build_library_2d(X_norm)
print(f"Library terms: {labels}")

In [ ]:
# Run SINDy with higher threshold for sparse noisy data
xi, iterations = sindy_stls(Theta, X_dot, threshold=0.2)
print(f"Converged in {iterations} iterations")

In [ ]:
# Display equations
var_names = ['H (hare)', 'L (lynx)']
print("Discovered equations (normalized):\n")
for i, var in enumerate(var_names):
    eq = format_equation(xi[i], labels, var_name=f"d{var}/dt")
    print(eq)

## Interpretation

The classic Lotka-Volterra predator-prey model:

- dH/dt = aH - bHL (prey grows, reduced by predation)
- dL/dt = -cL + dHL (predators die, grow from eating prey)

Look for interaction terms (xy) in the discovered equations.